In [6]:
import numpy as np
import normflowpy as nfp
import datasets
import torch
from matplotlib import pyplot as plt
from torch.distributions import MultivariateNormal
from experiments.functions import run_training

In [2]:
dataset_type = datasets.DatasetType.MOONS
n_training_samples = 50000
n_validation_samples = 10000
n_flow_blocks = 3
batch_size = 32
n_epochs = 50
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Current Working Device is set to:" + str(device))
training_data = datasets.get_dataset(dataset_type, n_training_samples)
training_dataset_loader = torch.utils.data.DataLoader(training_data, batch_size=batch_size,
                                                      shuffle=True, num_workers=0)

validation_data = datasets.get_dataset(dataset_type, n_validation_samples)
validation_dataset_loader = torch.utils.data.DataLoader(validation_data, batch_size=batch_size,
                                                        shuffle=False, num_workers=0)





Current Working Device is set to:cuda


# Create Glow Normalizing Flow Model

In [3]:
dim = training_data.dim()  # get data dim
base_distribution = MultivariateNormal(torch.zeros(dim, device=device),
                                       torch.eye(dim, device=device))  # generate a class for base distribution
flows = []
for i in range(n_flow_blocks):
    flows.append(
        nfp.flows.ActNorm(dim=dim))
    flows.append(
        nfp.flows.InvertibleFullyConnected(dim=dim))
    flows.append(
        nfp.flows.AffineCoupling(x_shape=[dim], parity=i % 2, net_class=nfp.base_nets.generate_mlp_class(), nh=32))
flow = nfp.NormalizingFlowModel(base_distribution, flows).to(device)

Starting Training Loop


  0%|          | 0/50 [00:00<?, ?it/s]

End Epoch with training loss:0.8529789548841563 and validtion loss:0.6852377616940215
End Epoch with training loss:0.625314123151551 and validtion loss:0.5498482723015184
End Epoch with training loss:0.5749874504727579 and validtion loss:0.519436664474658
End Epoch with training loss:0.5042935252494715 and validtion loss:0.47712750271105536
End Epoch with training loss:0.46690563599192325 and validtion loss:0.4494500444910397
End Epoch with training loss:0.4519591732659709 and validtion loss:0.4344318841402523
End Epoch with training loss:0.4406210657445117 and validtion loss:0.428698668464685
End Epoch with training loss:0.43800466336543487 and validtion loss:0.440927479832698
End Epoch with training loss:0.42571755149237867 and validtion loss:0.4086377393132962
End Epoch with training loss:0.41318915678534995 and validtion loss:0.3969904655656114
End Epoch with training loss:0.4118412152571474 and validtion loss:0.38973174836879343
End Epoch with training loss:0.4163065085083875 and 

# Set Optimizer and run training

In [ ]:
optimizer = torch.optim.Adam(flow.parameters(), lr=1e-4)
run_training(n_epochs, training_dataset_loader, validation_dataset_loader, flow, optimizer, device)

# Plot probability Map

In [4]:
def generate_probability_map(n_points, in_x_min, in_x_max, in_y_min, in_y_max, in_flow_model, in_device):
    results = []
    for x_tag in torch.linspace(in_x_min, in_x_max, n_points):
        _results_y = []
        for y_tag in torch.linspace(in_y_min, in_y_max, n_points):
            d = torch.stack([x_tag, y_tag]).reshape([1, -1]).to(in_device)
            _results_y.append(in_flow_model.nll(d).item())
        results.append(_results_y)
    return np.exp(-np.asarray(results)).T


x_min = -1.4
x_max = 2.1
y_min = -1
y_max = 1.5
res = generate_probability_map(200, x_min, x_max, y_min, y_max, flow, device)

fig, (ax0, ax1) = plt.subplots(2)
x = np.linspace(x_min, x_max, 200)
y = np.linspace(y_min, y_max, 200)

xx, yy = np.meshgrid(x, y)
im = ax0.pcolormesh(xx, yy, res)

ax1.plot(training_data[:, 0], training_data[:, 1], "o")
ax1.grid()
plt.show()